In [1414]:
import torch.nn as nn
import os

In [1415]:
print(os.getcwd())

/Users/aungmyinmoe/Documents/NUS-ISS GDipSA/SA60 Materials/SA4110 Machine Learning Applications/CA


In [1416]:
# define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5)
        # self.conv5 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=9*9*512, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=4)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.4)
        
        # # Define a projection for x1 to match the shape of x before addition
        # self.shortcut = nn.Sequential(
        #     nn.Conv2d(32, 64, kernel_size=5, stride=1),
        #     nn.MaxPool2d(kernel_size=2, stride=2),  # mimic the same pooling
        # )
        
    def forward(self, x):
        # convolution layer 1
        x1 = self.conv1(x)
        x = self.relu(x1)
        
        # convolution layer 2
        x = self.conv2(self.dropout(x))
        x = self.relu(x)
        x = self.pool(x)
        
        # convolution layer 3
        x = self.conv3(x)
        x = self.relu(x)
        
        # Match dimensions of x1
        # x1_proj = self.shortcut(x1)
        
        # convolution layer 4
        x = self.conv4(self.dropout(x))
        x = self.relu(x)
        x = self.pool(x)
        
        # convolution layer 5
        # x = self.conv5(x)
        # x = self.relu(x)
        
        # flattening
        x = x.view(-1, 9*9*512)
        
        # dense layer 1
        x = self.fc1(x)
        x = self.relu(x)
        
        # dense layer 1
        x = self.fc2(x)
        x = self.relu(x)
        
        # dense layer 2
        x = self.fc3(x)
        
        return x



In [1417]:
import torch.optim as optim

# instantiate the model, define the loss function and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005) # use Adam algo to optimize gradient descend

In [1418]:
import os
import numpy as np
import torch

#print(os.getcwd()) # find out current working directory in notebook

# prepare all the image file paths and labels
def load_filepaths(target_dir): 
    paths = []
    valid_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".gif")
    files = os.listdir(target_dir)
    for file in files:
        if file.endswith(valid_extensions):
            paths.append(f"{target_dir}/{file}")    # list all file names in the folder
    return paths

def prepare_data(target_dir):
    filepaths = []
    labels = []

    for i in range(4):
        fpaths = load_filepaths(target_dir + str(i))
        labels += [i] * len(fpaths)
        filepaths += fpaths # += add elements individually. append() will add the entire list

    return np.array(filepaths), torch.tensor(labels)

dir_train = "train/"
filepaths, labels = prepare_data(dir_train)

In [1419]:
from torchvision import transforms
from PIL import Image

# prepare to load images when testing
def load_images(filepaths):
    # Instantiate class to resize and transform image to tensor
    transform = transforms.Compose([
        transforms.Resize((48, 48)),
        # transforms.RandomResizedCrop(48, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
        transforms.ToTensor(),  # Convert PIL image to tensor (C x H x W)
        transforms.RandomHorizontalFlip(p=0.5),  # default is 0.5
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    ])

    tensor = None

    # List all files in the directory
    for item in filepaths:
        image = Image.open(item).convert("RGB")     # force 3 channels
        #print(f"image size = {image.size}")

        # transforms.ToTensor() performs transformations on images
        # values of img_tensor are in the range of [0.0, 1.0]
        img_tensor = transform(image) # convert into pytorch's tensor to work with
        #print(f"img_tensor.shape = {img_tensor.shape}")
        #input()

        if tensor is None:
            # size: [1,1,28,28]
            tensor = img_tensor.unsqueeze(0) # add a new dimension at specified index. in this case, added dimension is for batch dimension
        else:
            # concatenate becomes [2,1,28,28], [3,1,28,28], [4,1,28,28] ...
            # dim=0 concatenates along the axis=0 (row-wise)
            tensor = torch.cat((tensor, img_tensor.unsqueeze(0)), dim=0)

    return tensor

In [1420]:
# prepare to load images when training
def load_test_images(filepaths):
    # Instantiate class to resize and transform image to tensor
    transform = transforms.Compose([
        transforms.Resize((48, 48)),
        transforms.ToTensor(),  # Convert PIL image to tensor (C x H x W)
    ])

    tensor = None

    # List all files in the directory
    for item in filepaths:
        image = Image.open(item).convert("RGB")     # force 3 channels
        img_tensor = transform(image) # convert into pytorch's tensor to work with


        if tensor is None:
            # size: [1,1,28,28]
            tensor = img_tensor.unsqueeze(0) # add a new dimension at specified index. in this case, added dimension is for batch dimension
        else:
            # concatenate becomes [2,1,28,28], [3,1,28,28], [4,1,28,28] ...
            # dim=0 concatenates along the axis=0 (row-wise)
            tensor = torch.cat((tensor, img_tensor.unsqueeze(0)), dim=0)

    return tensor

In [1421]:
# train the model
# our hyper-parameters for training
n_epochs = 60
batch_size = 32

# model.train()
for epoch in range(n_epochs):
    model.train()
    # For tracking and printing our training-progress
    samples_trained = 0
    run_loss = 0
    correct_preds = 0
    total_samples = len(filepaths) 

    permutation = torch.randperm(total_samples) # perform permutation for each epoch so that the model does not over-fit or memorize
    for i in range(0, total_samples, batch_size):
        indices = permutation[i : i+batch_size]
        batch_inputs = load_images(filepaths[indices])
        batch_labels = labels[indices]

        # Forward pass: compute predicted outputs
        outputs = model(batch_inputs)

        # Compute loss
        loss = criterion(outputs, batch_labels)
        run_loss += loss.item() * len(batch_labels)

        # Backward pass and optimization step
        optimizer.zero_grad()
        loss.backward()     # compute gradients from back to front
        optimizer.step()    # update weights from computed gradients from back to front
        
        # Get probability-distributions
        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(probs, dim=1)  # store index of max probability (_ means ignore the value)

        # Calculate some stats
        # samples_trained += len(indices)
        samples_trained += len(batch_labels)
        avg_loss = run_loss / samples_trained

        correct_preds += torch.sum(preds == batch_labels) # compare predictions with labels
        accuracy = correct_preds / float(samples_trained) # cast to float to get "accuracy" in decimal 

        print(f"Epoch {epoch+1} " +
            f"({samples_trained}/{total_samples}): " +
            f"Loss={avg_loss:.5f}, Accuracy={accuracy:.5f}")

Epoch 1 (32/262): Loss=1.40839, Accuracy=0.28125
Epoch 1 (64/262): Loss=1.43407, Accuracy=0.32812
Epoch 1 (96/262): Loss=1.42482, Accuracy=0.26042
Epoch 1 (128/262): Loss=1.41432, Accuracy=0.28125
Epoch 1 (160/262): Loss=1.40595, Accuracy=0.28750
Epoch 1 (192/262): Loss=1.40484, Accuracy=0.26562
Epoch 1 (224/262): Loss=1.40255, Accuracy=0.27679
Epoch 1 (256/262): Loss=1.40610, Accuracy=0.26172
Epoch 1 (262/262): Loss=1.40578, Accuracy=0.25954
Epoch 2 (32/262): Loss=1.39050, Accuracy=0.34375
Epoch 2 (64/262): Loss=1.39373, Accuracy=0.31250
Epoch 2 (96/262): Loss=1.38938, Accuracy=0.27083
Epoch 2 (128/262): Loss=1.38799, Accuracy=0.28906
Epoch 2 (160/262): Loss=1.38824, Accuracy=0.28125
Epoch 2 (192/262): Loss=1.38958, Accuracy=0.28125
Epoch 2 (224/262): Loss=1.39491, Accuracy=0.25446
Epoch 2 (256/262): Loss=1.39415, Accuracy=0.25781
Epoch 2 (262/262): Loss=1.39446, Accuracy=0.25954
Epoch 3 (32/262): Loss=1.39988, Accuracy=0.21875
Epoch 3 (64/262): Loss=1.39969, Accuracy=0.20312
Epoch 3 

In [1422]:
# Test the model
dir_test = "test/" 
filepaths, labels = prepare_data(dir_test)

batch_size = 60
samples_tested = 0
correct_preds = 0
total_samples = len(filepaths)

model.eval()
for i in range(0, total_samples, batch_size):
    batch_inputs = load_test_images(filepaths[i : i + batch_size])
    batch_labels = labels[i : i + batch_size]

    # Forward pass: coyympute predicted outputs
    outputs = model(batch_inputs)

    # Get probability-distributions
    probs = torch.softmax(outputs, dim=1)
    _, preds = torch.max(probs, dim=1)

    # Determine accuracy
    samples_tested += len(batch_labels)
    correct_preds += torch.sum(preds == batch_labels)
    accuracy = correct_preds / float(samples_tested)

    print(f"({samples_tested}/{total_samples}): Accuracy={accuracy:.5f}")

(60/60): Accuracy=0.90000


In [1423]:
# torch.save(model, '90_accuracy_less_complex.pth')